## Chapter 12: Security from the Start

In a distributed application, security is not an afterthought—it must be designed in from the beginning. You need to protect sensitive data, secure communication between services, authenticate users, and authorize access to resources. .NET Aspire provides patterns and components to help you integrate security seamlessly into your application model.

In this chapter, you’ll learn how to:

- Secure service‑to‑service communication with mutual TLS (mTLS).
- Integrate an identity provider like Keycloak into your Aspire solution.
- Use the `Aspire.Keycloak` component to add authentication and authorization.
- Manage secrets securely using parameters and user secrets.
- Pass credentials to containers safely.
- Protect API endpoints with JWT tokens.

By the end, your e‑commerce application will have robust authentication and secure internal communication.

---

### 12.1 The Need for Security in Microservices

When you have multiple services communicating over a network, several threats emerge:

- **Eavesdropping**: An attacker could intercept network traffic and read sensitive data.
- **Tampering**: Data could be modified in transit.
- **Impersonation**: A malicious service could pretend to be a trusted service.
- **Unauthorized access**: Users or services might access resources they shouldn’t.

To address these, we use:

- **TLS/SSL** for encryption and authentication of communication.
- **mTLS** for mutual authentication between services.
- **OAuth2 / OpenID Connect** for user authentication and authorization.
- **JWT tokens** to convey identity and claims.

Aspire helps by providing hosting packages for identity servers and making it easy to inject configuration.

---

### 12.2 Securing Service-to-Service Communication with mTLS

By default, when you add a project reference, Aspire injects environment variables for both HTTP and HTTPS endpoints. However, the communication between services is typically over HTTP (unencrypted) unless you explicitly use HTTPS. In production, you should always use HTTPS, and for service-to-service, you may want **mutual TLS** where both client and server present certificates.

#### 12.2.1 Enforcing HTTPS Between Services

In your service defaults, you can configure `HttpClient` to prefer HTTPS by setting the base address to `https://servicename`. But the service must have an HTTPS endpoint. Aspire automatically exposes HTTPS endpoints for projects (via the `applicationUrl` in launchSettings.json). However, by default, those HTTPS endpoints use self‑signed certificates, which may not be trusted.

For local development, you can trust the ASP.NET Core development certificate. In production, you'd use proper certificates.

To ensure that a service only accepts HTTPS, you can configure it to redirect HTTP to HTTPS. In the API service `Program.cs`:

```csharp
if (!app.Environment.IsDevelopment())
{
    app.UseHttpsRedirection();
}
```

But for service‑to‑service, you might not want redirection; you want the client to use HTTPS directly.

#### 12.2.2 Setting Up mTLS

mTLS requires certificates on both sides. This is complex to set up manually, but Aspire can help by generating and injecting certificates into containers. The `Aspire.Hosting` package includes support for mTLS via the `WithMutualTls` extension.

In the AppHost, you can enable mTLS for a resource:

```csharp
var apiService = builder.AddProject<Projects.MyAspireApp_ApiService>("apiservice")
    .WithMutualTls();  // This generates a certificate and injects it into the service
```

When you call `WithReference` from another service, Aspire will automatically configure that service to trust the certificate and present its own client certificate. This requires that both services have the mTLS feature enabled.

Under the hood, Aspire uses a certificate authority (CA) that it creates and trusts. The CA certificate is injected into the container’s trusted root store. Then each service gets a client certificate signed by that CA.

To make this work, your service must be configured to require client certificates. In the API service `Program.cs`:

```csharp
builder.WebHost.ConfigureKestrel(options =>
{
    options.ConfigureHttpsDefaults(httpsOptions =>
    {
        httpsOptions.ClientCertificateMode = ClientCertificateMode.RequireCertificate;
        httpsOptions.AllowAnyClientCertificate = false; // Require valid cert from our CA
    });
});
```

And in the web frontend, when creating an `HttpClient`, you need to ensure it uses the client certificate. Aspire’s `CreateHttpClient` extension automatically does this when mTLS is enabled.

**Note**: mTLS support is evolving in Aspire; check the latest documentation for the exact APIs.

---

### 12.3 Integrating Keycloak for Authentication and Authorization

Keycloak is an open‑source identity and access management solution. It supports OpenID Connect, OAuth2, and SAML. Aspire provides a hosting package to run Keycloak in a container, and a client component to connect to it.

#### 12.3.1 Adding Keycloak to the AppHost

First, install the hosting package:

```bash
cd MyAspireApp.AppHost
dotnet add package Aspire.Hosting.Keycloak
```

Now add a Keycloak resource. For local development, you can run it as a container:

```csharp
var keycloak = builder.AddKeycloak("keycloak", port: 8080)
    .WithImageTag("22.0")
    .WithEnvironment("KC_BOOTSTRAP_ADMIN_USERNAME", "admin")
    .WithEnvironment("KC_BOOTSTRAP_ADMIN_PASSWORD", "admin")
    .WithDataVolume();  // persist realm data
```

This starts Keycloak with default admin credentials. In production, you'd use secure passwords and possibly integrate with a database.

You also need to create a **realm** and a client for your application. You can do this manually via the Keycloak admin console, or automate it using the Keycloak REST API. Aspire doesn't provide a built‑in way to configure the realm, but you can add a container that runs a realm import on startup. For simplicity, we'll do it manually for now.

#### 12.3.2 Configuring Keycloak

Start the AppHost, navigate to `http://localhost:8080` (or the port you configured), and log in with `admin/admin`. Then:

1. Create a new realm called `ecommerce`.
2. In that realm, create a client:
   - Client ID: `ecommerce-api`
   - Access Type: `confidential` (for service‑to‑service) or `public` for SPAs.
   - Valid Redirect URIs: `*` (for development).
   - Service Accounts Enabled: On (if you want client credentials flow).
3. Note the client secret (if confidential).

For user authentication, you might also create a user (e.g., `testuser` with password). We'll use that to test.

#### 12.3.3 Adding the Keycloak Component to the API

Now in the API service project, add the Keycloak component:

```bash
cd ../MyAspireApp.ApiService
dotnet add package Aspire.Keycloak.Authentication
```

But note: there is no official `Aspire.Keycloak.Authentication` package as of now. Instead, we'll use the standard `Microsoft.AspNetCore.Authentication.JwtBearer` and configure it manually, using the connection information from the AppHost. However, Aspire provides an authentication component that integrates with the .NET authentication stack. For Keycloak, you would typically configure OpenID Connect.

Let's use the standard JWT Bearer authentication, pointing to the Keycloak issuer. We'll need the Keycloak URL and realm information, which we can get via environment variables injected from the AppHost.

First, in the AppHost, reference Keycloak from the API service:

```csharp
apiService.WithReference(keycloak);
```

This injects environment variables like `ConnectionStrings__keycloak`? Actually, Keycloak is a container with HTTP endpoints, so `WithReference` will inject the service URL as `services__keycloak__http__0`. We can use that in the API to construct the issuer URL.

In the API's `Program.cs`, add authentication:

```csharp
using Microsoft.AspNetCore.Authentication.JwtBearer;
using Microsoft.IdentityModel.Tokens;

var keycloakUrl = builder.Configuration.GetServiceUri("keycloak")?.ToString() ?? "http://localhost:8080";
var realm = "ecommerce"; // Should come from configuration

builder.Services.AddAuthentication(JwtBearerDefaults.AuthenticationScheme)
    .AddJwtBearer(options =>
    {
        options.Authority = $"{keycloakUrl}/realms/{realm}";
        options.Audience = "ecommerce-api"; // client id
        options.RequireHttpsMetadata = false; // only for development
        options.TokenValidationParameters = new TokenValidationParameters
        {
            ValidateIssuer = true,
            ValidIssuer = $"{keycloakUrl}/realms/{realm}",
            ValidateAudience = true,
            ValidAudience = "ecommerce-api",
            ValidateLifetime = true
        };
    });

builder.Services.AddAuthorization();
```

Also add the authentication and authorization middleware in the app:

```csharp
app.UseAuthentication();
app.UseAuthorization();
```

Now protect an endpoint with `[Authorize]`:

```csharp
app.MapGet("/protected", () => "This is protected").RequireAuthorization();
```

#### 12.3.4 Testing with a Token

To test, you need to obtain a token from Keycloak. You can use the password grant (for user) or client credentials (for service). For example, using `curl`:

```bash
curl -X POST http://localhost:8080/realms/ecommerce/protocol/openid-connect/token \
  -H "Content-Type: application/x-www-form-urlencoded" \
  -d "client_id=ecommerce-api" \
  -d "client_secret=your-client-secret" \
  -d "grant_type=password" \
  -d "username=testuser" \
  -d "password=testpassword"
```

This returns an access token. Then call the protected endpoint:

```bash
curl -H "Authorization: Bearer <token>" http://apiservice/protected
```

#### 12.3.5 Using Client Credentials for Service-to-Service

For service‑to‑service calls, you might want the web frontend to authenticate to the API using client credentials. The web frontend can acquire a token using its own client ID and secret, and attach it to outgoing HTTP requests.

In the web frontend, you can use `HttpClient` with a custom handler that adds the token. You'll need to implement a token cache and refresh logic.

---

### 12.4 Managing Secrets Securely

Secrets like the Keycloak client secret must be stored securely. In the AppHost, you can use parameters to pass them to resources.

#### 12.4.1 Using Parameters for Client Secrets

Define a parameter for the Keycloak client secret:

```csharp
var keycloakClientSecret = builder.AddParameter("keycloak-client-secret", secret: true);
```

Then, when adding the Keycloak resource, you can pass it as an environment variable or use it when referencing. However, the client secret is not needed by the Keycloak container itself; it's needed by the API service. So we'll pass it to the API via an environment variable.

```csharp
apiService.WithEnvironment("Keycloak__ClientSecret", keycloakClientSecret);
```

In the API's `appsettings.json`, you can have a placeholder, and the environment variable will override it.

Alternatively, you can use the `AddJwtBearer` options to read from configuration:

```csharp
options.TokenValidationParameters.ValidAudience = builder.Configuration["Keycloak:ClientId"];
```

And set `Keycloak:ClientSecret` from the environment variable.

#### 12.4.2 User Secrets for Local Development

For local development, you can store the parameter value in user secrets:

```bash
cd MyAspireApp.AppHost
dotnet user-secrets init
dotnet user-secrets set "Parameters:keycloak-client-secret" "your-client-secret"
```

Now when you run the AppHost, the secret will be passed to the API service.

---

### 12.5 Securing Containers with Credentials

When you add a container like Keycloak, you need to set the admin password. This is also a secret. Use a parameter:

```csharp
var keycloakAdminPassword = builder.AddParameter("keycloak-admin-password", secret: true);

var keycloak = builder.AddKeycloak("keycloak", port: 8080)
    .WithEnvironment("KC_BOOTSTRAP_ADMIN_PASSWORD", keycloakAdminPassword)
    .WithDataVolume();
```

Store the password in user secrets.

---

### 12.6 Hands‑on: Add Keycloak Authentication to the E‑Commerce API

Let's walk through adding Keycloak to our e‑commerce application and protecting the product endpoints.

#### Step 1: Add Keycloak to AppHost

Add the package and the resource as shown above. Use parameters for admin password and client secret.

#### Step 2: Configure Keycloak Realm and Client

Run the AppHost, open Keycloak admin console, create realm `ecommerce`, client `ecommerce-api` with `confidential`, service accounts enabled, and set valid redirect URIs to `*` (for development). Note the client secret.

#### Step 3: Add Authentication to API Service

Install the JWT bearer package:

```bash
dotnet add package Microsoft.AspNetCore.Authentication.JwtBearer
```

Update `Program.cs` as described, reading the Keycloak URL from service discovery and client secret from environment.

#### Step 4: Protect the Product Endpoints

Add `[Authorize]` to the product endpoints, or use policy‑based authorization.

#### Step 5: Obtain a Token and Test

Use Postman or curl to get a token and call the protected endpoint.

#### Step 6: (Optional) Add Token Acquisition to Web Frontend

The web frontend needs to authenticate users. You can use OpenID Connect middleware to redirect to Keycloak for login. Add the Microsoft authentication packages:

```bash
dotnet add package Microsoft.AspNetCore.Authentication.OpenIdConnect
```

In the web frontend `Program.cs`:

```csharp
builder.Services.AddAuthentication(options =>
{
    options.DefaultScheme = CookieAuthenticationDefaults.AuthenticationScheme;
    options.DefaultChallengeScheme = OpenIdConnectDefaults.AuthenticationScheme;
})
.AddCookie()
.AddOpenIdConnect(options =>
{
    options.Authority = $"{keycloakUrl}/realms/ecommerce";
    options.ClientId = "ecommerce-web"; // you need a separate client for the web app
    options.ClientSecret = builder.Configuration["Keycloak:WebClientSecret"];
    options.ResponseType = "code";
    options.SaveTokens = true;
    options.GetClaimsFromUserInfoEndpoint = true;
    options.Scope.Add("profile");
    options.Scope.Add("email");
});
```

Then you can use the `[Authorize]` attribute on Razor pages or components, and access the token to call the API.

---

### 12.7 Summary

In this chapter, you learned how to secure your Aspire application:

- mTLS provides mutual authentication between services, ensuring only trusted services can communicate.
- Keycloak integration adds OpenID Connect and OAuth2 support for user authentication and authorization.
- Parameters and user secrets keep sensitive information out of source code.
- Environment variables injected by the AppHost convey connection details and secrets to services.

With these techniques, your application is now protected against common security threats. In the next chapter, we’ll move on to **Deployment Fundamentals with Aspire**, where you’ll learn how to take your application from development to production using Kubernetes and the `aspirate` tool.

---

**Exercises**

1. Extend the API to require a specific scope (e.g., `product:read`) on the product endpoints. Configure Keycloak to assign this scope to the client and test.
2. Implement token refresh in the web frontend so the user stays logged in.
3. Set up mTLS between the API and the worker service. Use the `WithMutualTls` extension and configure Kestrel accordingly.
4. Write an integration test that verifies that an unauthenticated request to a protected endpoint returns 401.

In Chapter 13, we’ll explore **Deployment Fundamentals with Aspire**, covering the `aspirate` tool and Kubernetes manifests.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='11. testing_aspire_applications.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../4. from_development_to_production/13. deployment_fundamentals_with_aspire.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
